# Wellness 심리 상담 데이터에 대한 KoBERT 학습 Question & Answer 
데이터와 클래스 셋으로 이루어져, 질의 데이터가 들어왔을 때, Answer 클래스를 예측하도록 학습

## 1.Google Drive 연동
- 모델 파일과 학습 데이터가 저장 되어있는 구글 드라이브의 디렉토리와 Colab을 연동.  
- 좌측상단 메뉴에서 런타임-> 런타임 유형 변경 -> 하드웨어 가속기 -> GPU 선택 후 저장

### 1.1 GPU 연동 확인

In [1]:
!nvidia-smi

Sun Jul 12 15:02:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 1.2 Google Drive 연동
아래 코드를 실행후 나오는 URL을 클릭하여 나오는 인증 코드 입력

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Colab 디렉토리 아래 dialogLM 경로 확인**




In [3]:
!ls drive/'My Drive'/'Colab Notebooks'/

 BERT_X		      'fastprogress example.ipynb'    NarrativeKoGPT2
 Data		       KorQuAD-beginner		      ReforBERT
 dialogLM	       korquad-finetuing.ipynb
 EnlipleBERTFintuing   korquad-finetuing-ver2.ipynb


**필요 패키지 설치**

In [4]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/dialogLM/requirements.txt

     |████████████████████████████████| 675kB 4.0MB/s 
     |████████████████████████████████| 3.0MB 15.2MB/s 
     |████████████████████████████████| 1.1MB 30.6MB/s 
     |████████████████████████████████| 890kB 40.3MB/s 
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251597 sha256=38fccb9861fa80a1a589ecbbe42b65047a08572abec6f19450685692c3210e3c
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6c1b256bef0d69f5b6da3093565f3e172f233e84cc2c8cdf6104dd17d6f16de3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built kss sacremoses
ERROR: transformers 2.11.0 has requirement tokenizers==0.7.0, but you'll have tokenizers 0.8.0 which is incompatible.


## KoBERT QA Training

**Path 추가**

In [5]:
import sys
sys.path.append('drive/My Drive/Colab Notebooks/')
sys.path.append('drive/My Drive/Colab Notebooks/dialogLM')

### 2.1 import package

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from tqdm import tqdm

import torch
from transformers import AdamW
from torch.utils.data import dataloader
from dialogLM.dataloader.wellness import WellnessTextClassificationDataset
from dialogLM.model.kobert import KoBERTforSequenceClassfication

In [7]:
torch.cuda.is_available()

True

**Train 함수**

In [8]:
def train(device, epoch, model, optimizer, train_loader, save_step, save_ckpt_path, train_step = 0):
    losses = []
    train_start_index = train_step+1 if train_step != 0 else 0
    total_train_step = len(train_loader)
    model.train()

    with tqdm(total= total_train_step, desc=f"Train({epoch})") as pbar:
        pbar.update(train_step)
        for i, data in enumerate(train_loader, train_start_index):

            # data.to(device)
            optimizer.zero_grad()
            outputs = model(**data)

            loss = outputs[0]

            losses.append(loss.item())

            loss.backward()
            optimizer.step()

            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")

            if i >= total_train_step or i % save_step == 0:
                torch.save({
                    'epoch': epoch,  # 현재 학습 epoch
                    'model_state_dict': model.state_dict(),  # 모델 저장
                    'optimizer_state_dict': optimizer.state_dict(),  # 옵티마이저 저장
                    'loss': loss.item(),  # Loss 저장
                    'train_step': i,  # 현재 진행한 학습
                    'total_train_step': len(train_loader)  # 현재 epoch에 학습 할 총 train step
                }, save_ckpt_path)

    return np.mean(losses)

### KoBERT Question & Answer Training for Wellness dataset

In [ ]:
root_path='drive/My Drive/Colab Notebooks/dialogLM'
data_path = f"{root_path}/data/wellness_dialog_for_text_classification_train.txt"
checkpoint_path =f"{root_path}/checkpoint"
save_ckpt_path = f"{checkpoint_path}/kobert-wellnesee-text-classification.pth"

n_epoch = 50          # Num of Epoch
batch_size = 16      # 배치 사이즈
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)
save_step = 100 # 학습 저장 주기
learning_rate = 5e-6  # Learning Rate

# WellnessTextClassificationDataset 데이터 로더
dataset = WellnessTextClassificationDataset(file_path=data_path, device=device)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = KoBERTforSequenceClassfication()
model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
      'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

pre_epoch, pre_loss, train_step = 0, 0, 0
if os.path.isfile(save_ckpt_path):
    checkpoint = torch.load(save_ckpt_path, map_location=device)
    pre_epoch = checkpoint['epoch']
    # pre_loss = checkpoint['loss']
    train_step =  checkpoint['train_step']
    total_train_step =  checkpoint['total_train_step']

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}")  #, loss={pre_loss}\n")
    # best_epoch += 1

losses = []
offset = pre_epoch
for step in range(n_epoch):
    epoch = step + offset
    loss = train(device, epoch, model, optimizer, train_loader, save_step, save_ckpt_path, train_step)
    losses.append(loss)

# data
data = {
    "loss": losses
}
df = pd.DataFrame(data)
display(df)

# graph
plt.figure(figsize=[12, 4])
plt.plot(losses, label="loss")
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

Train(25):   0%|          | 0/296 [00:00<?, ?it/s]

load pretrain from: drive/My Drive/Colab Notebooks/dialogLM/checkpoint/kobert-wellnesee-text-classification.pth, epoch=25


Train(25): 396it [48:58,  7.42s/it, Loss: 1.281 (2.581)]
Train(26): 396it [49:03,  7.43s/it, Loss: 0.828 (2.487)]
Train(27): 396it [49:05,  7.44s/it, Loss: 2.565 (2.389)]
Train(28): 396it [49:02,  7.43s/it, Loss: 2.982 (2.305)]
Train(29): 396it [49:02,  7.43s/it, Loss: 2.821 (2.206)]
Train(30): 396it [49:05,  7.44s/it, Loss: 2.402 (2.119)]
Train(31): 396it [49:05,  7.44s/it, Loss: 2.044 (2.030)]
Train(32): 396it [49:05,  7.44s/it, Loss: 2.681 (1.944)]
Train(33): 396it [49:04,  7.44s/it, Loss: 2.491 (1.864)]
Train(34): 396it [48:55,  7.41s/it, Loss: 1.737 (1.786)]
Train(35): 396it [49:02,  7.43s/it, Loss: 1.816 (1.702)]
Train(36): 396it [49:06,  7.44s/it, Loss: 1.276 (1.620)]
Train(37): 396it [49:10,  7.45s/it, Loss: 1.464 (1.547)]
Train(38): 396it [49:04,  7.44s/it, Loss: 2.667 (1.478)]
Train(39): 396it [49:05,  7.44s/it, Loss: 0.806 (1.406)]
Train(40): 396it [49:02,  7.43s/it, Loss: 1.478 (1.337)]
Train(41): 396it [49:09,  7.45s/it, Loss: 2.045 (1.267)]
Train(42): 396it [49:04,  7.44s